In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
from torchvision import transforms,models
from torch import nn, optim
import torch.nn.functional as F
from torch.autograd import Variable

from sklearn.model_selection import train_test_split

import os

In [ ]:
train_sample = pd.read_csv("../input/digit-recognizer/train.csv", dtype=np.float32)
test_sample = pd.read_csv("../input/digit-recognizer/test.csv", dtype=np.float32)
samp_submission = pd.read_csv("../input/digit-recognizer/sample_submission.csv")
train_sample.info()

In [ ]:
train_sample.label.head()

In [ ]:
targets_in_np = train_sample.label.values
features_in_np = train_sample.loc[:, train_sample.columns!='label'].values/255 #normalisation

In [ ]:
features_train_np, features_test_np, targets_train_np, targets_test_np = train_test_split(features_in_np, targets_in_np, test_size=0.15) 

In [ ]:
features_train=torch.from_numpy(features_train_np)
targets_train=torch.from_numpy(targets_train_np).type(torch.LongTensor)

features_test=torch.from_numpy(features_test_np)
targets_test=torch.from_numpy(targets_test_np).type(torch.LongTensor)

In [ ]:
BATCH_SIZE = 256

train = torch.utils.data.TensorDataset(features_train,targets_train)
test = torch.utils.data.TensorDataset(features_test,targets_test)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = True)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = True)

In [ ]:
def visualize_image(data, index, pred=False, val=0):
    '''This funtion can be used to visualize the images'''
    plt.imshow(data[index].reshape(28,28))
    plt.axis("off")
    plt.title("Handwritten Digit Image")
    plt.show()
visualize_image(features_in_np, 1)

In [ ]:
features_train.shape

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 10)
        
        self.dropout = nn.Dropout(p=0.25)
        
        self.log_softmax = F.log_softmax
        
    def forward(self, x):
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.dropout(F.relu(self.fc4(x)))
        
        x = self.log_softmax(self.fc5(x),dim=1)
        return x

In [ ]:
model = Classifier()

loss_criteria = nn.NLLLoss()

optimizer = optim.Adam(model.parameters(), lr=0.0013)

epochs = 25
steps = 0
cnt = 50 

train_losses, test_losses = [], []

for i in range(epochs):
    current_loss=0
    for image, labels in train_loader :
        steps+=1
        
        optimizer.zero_grad()
        
        log_prob = model(image)
        loss = loss_criteria(log_prob,labels)
        
        loss.backward()
        optimizer.step()
        
        current_loss += loss.item()
        if steps % cnt == 0:
            curr_loss = 0
            accuracy = 0;
            
            with torch.no_grad():
                model.eval()
                for image,labels in test_loader : 
                    log_prob = model(image)
                    curr_loss += loss_criteria(log_prob, labels)
                    prob = torch.exp(log_prob)
                    
                    max_val, max_class = prob.topk(1,dim=1)
                    is_equal = max_class== labels.view(*max_class.shape)
                    accuracy += torch.mean(is_equal.type(torch.FloatTensor))
                    
                model.train()
                
                train_losses.append(current_loss/len(train_loader))
                test_losses.append(curr_loss/len(test_loader))
                
                print("Epoch: {}/{}.. ".format(i+1, epochs),
                  "Training Loss: {:.3f}.. ".format(train_losses[-1]),
                  "Test Loss: {:.3f}.. ".format(test_losses[-1]),
                  "Test Accuracy: {:.3f}".format(accuracy/len(test_loader)))
            

In [ ]:


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
plt.legend(frameon=False)



In [ ]:
def view_classify(img, ps):
    ''' Function for viewing an image and it's predicted classes.
    '''
    ps = ps.data.numpy().squeeze()

    fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
    ax1.imshow(img.resize_(1, 28, 28).numpy().squeeze())
    ax1.axis('off')
    ax2.barh(np.arange(10), ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
    ax2.set_yticklabels(np.arange(10))
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()


In [ ]:
%matplotlib inline
def make_prediction(data):
    images, labels = next(iter(data))

    img = images[42].view(1, 784)
    # Turn off gradients to speed up this part
    with torch.no_grad():
        logps = model(img)

    # Output of the network are log-probabilities, need to take exponential for probabilities
    ps = torch.exp(logps)
    #view_classify(img.view(1, 28, 28), ps)
make_prediction(test_loader)

In [ ]:
test_in_np = test_sample.values/255
test_final = torch.from_numpy(test_in_np)

In [ ]:
# Creating fake labels for convenience of passing into DataLoader
fake_labels = np.zeros(test_in_np.shape)
fake_labels = torch.from_numpy(fake_labels)

In [ ]:
submission_final_data = torch.utils.data.TensorDataset(test_final, fake_labels)

submission_loader = torch.utils.data.DataLoader(submission_final_data,batch_size = BATCH_SIZE, shuffle = False)

In [ ]:
make_prediction(submission_loader)

In [ ]:
submission = [['ImageId', 'Label']]

with torch.no_grad():
    model.eval()
    image_id = 1
    for images, _ in submission_loader:
        log_prob = model(images)
        prob = torch.exp(log_prob)
        top_val, top_class = prob.topk(1,dim=1)
        
        for prediction in top_class : 
            submission.append([image_id,prediction.item()])
            image_id+=1

In [ ]:
submission_df = pd.DataFrame(submission)
submission_df.columns = submission_df.iloc[0]
submission_df = submission_df.drop(0, axis=0)

In [ ]:
submission_df.to_csv("submission.csv", index=False)
